# Estudio sobre la tienda on-line de:

## [AMANTIS](https://www.amantis.net/)

## ¿Por qué este estudio?

Primero, este es un trabajo de análisis de datos y de predicción sobre una tienda on-line. Creo que las personas que veamos esta documentación somos adultas y lo suficientemente maduras para asumir que el sexo es parte de nuestra vida.

Por este motivo, el hacer un estudio sobre una tienda on-line erótica es lo mismo que realizarlo sobre una tienda de ropa, muebles, un supermercado,...

Segundo, yo soy usuario registrado de esta tienda on-line. No solo realizo compras para mí o para mi pareja sino también para otras personas. 
Es por este motivo el que recibo periódicamente correos de ofertas de esta tienda on-line y es aquí en donde entra mi motivación personal.

Los correos que envían a sus usuarios están basados en ofertas que lanzan (estas ofertas están basadas en rebajas) y son muy arbitrarias. Suelo recibir correos de ofertas de determinados productos de los que NO he realizado una compra del mismo o similares. 

Por este motivo, considero interesante poder realizar algún tipo de estudio sobre los productos que tengan en la tienda y poder dar una mejor experiencia al comprador.

## Consideraciones para el proyecto.

Este proyecto está estructurado en 4 partes:

- Obtención y tratamiento de datos de la página web, a partir de *Webscrapping* y su manipulación para poder tratarlos a través de las librerías de *pandas* y *numpy*. 
   - A su vez se realizará algún tipo de visionado de los datos obtenidos para ver diversos aspectos a tener en cuenta.
- Generación de nuevas variables, *featuring engineering*, para poder dar una mejor visión de los productos que ofrecen en la tienda.
    - A través de la generación de estas nuevas variables, pretendemos dar un visionario más completo sobre la relación de las diversas variables entre sí y los usuarios.
    - Estas variables pueden ser obtenidas a través de *pandas*, *RegEx* y *NLP*.
- Análisis de la información obtenida y generada para ver la relación existente entre usuarios, productos, fechas...
   - Esta información será aplicada con las librerías de *pandas*, *matplotlib* y *seaborn*.
- Generación de modelados de los datos obtenidos para establecer gustos de los usuarios y dar sugerencia de otros productos que tengan etiquetas similares.

    


## 1. Obtención y tratamiento de los datos.

A partir de la **URL** *'https://www.amantis.net/productos-amantis/'* vamos a realizar un recorrido por las diferentes páginas que dispone y en donde están alojadas las **URLs** de los diferentes productos que disponen.

Una vez que accedemos a la dirección de cada producto, vemos que hay una serie de datos que queremos recoger de la misma:
> Nombre.
> Precio del producto.
>
> Precio rebajado del producto.
>
> Descripción e información sobre el producto (tamaño, duración de baterias, limpieza ...). 
>
> Comentarios.

Dentro de comentarios disponemos de más información, que consideramos interesante:
> Usuario.
>
> Rating.
>
> Fecha comentario.
>
> Cuerpo del comentario.


A través de la librería para Webscrapping *Beautiful Soap* procederemos a extraer esta información y en determinados casos a tratarla para poder trabajar con ella.

Para ello cargaremos las librerías necesarias.

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
# pages= np.arange(1,5)
pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []
name=[]
regular_prices=[]
new_price=[]
info=[]
user_comments=[]
comment=[]
date=[]
ratings=[]
id=[]
comentarios=[]



'''Generamos 2 diccionarios con los datos importantes para ingresar en una BBDD'''

diccionario_datos_productos={"ID":id,"NAME":name,"INFO":info,"LISTA_URL":lista_URLs,"REGULAR_PRICE":regular_prices,"DISCOUNT_PRICE":new_price}

diccionario_comentarios_productos={"ID":id,"COMENTARIOS":comentarios}



''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)


for i in range(len(lista_URLs)):
    id.append(i)

    
'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True)
        name.append(nombre)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:]
    documentation = ''.join(information)
    info.append(documentation)


    '''Vamos a obtener los datos de los comentarios de los usuarios'''

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    user_comments.append(user_comments_product)

    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)
    date.append(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    comment.append(comments_product)

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)
    ratings.append(rating)

    datos = list(zip( date_comments_product,rating, user_comments_product,comments_product ))
    comentarios.append(datos)

for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]

'''Generamos un dataframe con este diccionario para poder trabajar con los mismos y generar una nueva estructura para poder trabajar con ella'''
comentarios_productos=pd.DataFrame(diccionario_comentarios_productos)

id=[]
comments=[]
date=[]
ratio=[]
users=[]
comment=[]

comentarios=pd.DataFrame()
diccionario={"id":id,"comments":comments}

for id_product,n_comments in enumerate (comentarios_productos['COMENTARIOS']):
    # print("Imprimiendo texto del indice",id_product)
    # print("Imprimiento n_comentarios",len(n_comments))
    for i in n_comments:
        # print("id",id_product,"coments",comments)
        id.append(id_product)
        comments.append(i)


for j in range(len(diccionario['comments'])):
    date.append(diccionario['comments'][j][0])
    ratio.append(diccionario['comments'][j][1])
    users.append(diccionario['comments'][j][2])
    comment.append(diccionario['comments'][j][3])


comentarios['ID']=pd.Series(id)
comentarios['DATE']=pd.Series(date)
comentarios['RATIO']=pd.Series(ratio)
comentarios['USERS']=pd.Series(users)
comentarios['COMMENT']=pd.Series(comment)
comentarios.info()

Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20
Pagina 21
Pagina 22
Pagina 23
Pagina 24
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11375 entries, 0 to 11374
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       11375 non-null  int64 
 1   DATE     11375 non-null  object
 2   RATIO    11375 non-null  int64 
 3   USERS    11375 non-null  object
 4   COMMENT  11375 non-null  object
dtypes: int64(2), object(3)
memory usage: 444.5+ KB


Vamos a visualizar los datos que existen en los dos diccionarios.

Para ello pasaremos el diccionario de productos a un Dataframe y después haremos un estudio de los mismos.

Vamos a ver si cuántos duplicados hay en este Dataframe, ya que navegando por la web hemos visto algún producto repetido.

In [34]:
productos=pd.DataFrame(diccionario_datos_productos)
productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              576 non-null    int64  
 1   NAME            576 non-null    object 
 2   INFO            576 non-null    object 
 3   LISTA_URL       576 non-null    object 
 4   REGULAR_PRICE   576 non-null    float64
 5   DISCOUNT_PRICE  576 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 27.1+ KB


In [5]:
comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11375 entries, 0 to 11374
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       11375 non-null  int64 
 1   DATE     11375 non-null  object
 2   RATIO    11375 non-null  int64 
 3   USERS    11375 non-null  object
 4   COMMENT  11375 non-null  object
dtypes: int64(2), object(3)
memory usage: 444.5+ KB


In [27]:
productos.NAME.value_counts()

TOK ANAL GROSSO - Vibrador anal con bala extraible                  2
JUSTISSE CHOCKER, unisex con argolla                                2
TOTEM, la anilla para pene de amantis                               2
DADOS AMANTIS, jugar nunca fue tan divertido                        2
MONAMOUR, plug anal con brillante de corazón                        2
                                                                   ..
Mordaza con bola maciza de suave silicona roja o negra              1
Plumi-pendientes amantis para pezones                               1
TENTA, Masajeador rotador con vibración                             1
Glup! Dildo LIZO de silicona, dos tamaños y compatible con arnés    1
Conjunto de encaje + collar y muñequeras con cadenas                1
Name: NAME, Length: 555, dtype: int64

#### Eliminando duplicados de los dos dataframes.

In [35]:
productos_clean = productos.drop_duplicates(subset='NAME', keep='first')
productos_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555 entries, 0 to 575
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              555 non-null    int64  
 1   NAME            555 non-null    object 
 2   INFO            555 non-null    object 
 3   LISTA_URL       555 non-null    object 
 4   REGULAR_PRICE   555 non-null    float64
 5   DISCOUNT_PRICE  555 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 30.4+ KB


In [29]:
removed_id = productos[productos.duplicated(subset='NAME', keep='first')]['ID']
comentarios_clean = comentarios[~comentarios['ID'].isin(productos[productos['ID'].isin(removed_id)]['ID'])]
comentarios_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10333 entries, 0 to 11374
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       10333 non-null  int64 
 1   DATE     10333 non-null  object
 2   RATIO    10333 non-null  int64 
 3   USERS    10333 non-null  object
 4   COMMENT  10333 non-null  object
dtypes: int64(2), object(3)
memory usage: 484.4+ KB


#### Generamos nuevas variables dentro del dataframe de **productos**.

Vamos a crear 2 nuevas columnas a partir de *Name* y *Description*, donde dejaremos el nombre del producto y su slogan por un lado y por otro la descripción y las características por otro.

En principio son variables que no necesitaremos para estos estudios pero las guardaremos por si hay que hacer algún estudio posterior de las mismas (comparativas de productos similares de esta página web o de otras páginas web, por ejemplo).

In [36]:
productos_clean['NAME'] = productos_clean['NAME'].str.replace(r'-(?=\w)', '_')
productos_clean[['PRODUCT', 'SLOGAN']] = productos_clean['NAME'].str.split('[,-.]', 1, expand=True)
productos_clean['PRODUCT'] = productos_clean['PRODUCT'].str.strip()
productos_clean['SLOGAN'] = productos_clean['SLOGAN'].str.strip()
productos_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555 entries, 0 to 575
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              555 non-null    int64  
 1   NAME            555 non-null    object 
 2   INFO            555 non-null    object 
 3   LISTA_URL       555 non-null    object 
 4   REGULAR_PRICE   555 non-null    float64
 5   DISCOUNT_PRICE  555 non-null    float64
 6   PRODUCT         555 non-null    object 
 7   SLOGAN          460 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 39.0+ KB


C:\Users\Javier\AppData\Local\Temp\ipykernel_10948\1895620206.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  productos_clean['NAME'] = productos_clean['NAME'].str.replace(r'-(?=\w)', '_')
C:\Users\Javier\AppData\Local\Temp\ipykernel_10948\1895620206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  productos_clean['NAME'] = productos_clean['NAME'].str.replace(r'-(?=\w)', '_')
C:\Users\Javier\AppData\Local\Temp\ipykernel_10948\1895620206.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  productos_clean[['PRODUCT', 'SLOGAN']] = productos_clean['NAME'].str.split('[,-.]', 1, expand=True)
C:\Users\Javier\

Vamos a realizar un visionado de los datos que tienen para discernir donde se puede realizar la separación.

Por un lado extraeremos las características en una nueva columna, dejando la Description en la misma. 

In [37]:
productos_clean['CHARACTERISTICS'] = productos_clean['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
productos_clean['DESCRIPTION'] = productos_clean['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()
productos_clean['CHARACTERISTICS'] = productos_clean['CHARACTERISTICS'].str.replace('\r', ' ')
productos_clean['DESCRIPTION'] = productos_clean['DESCRIPTION'].str.replace('\r', ' ')
productos_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555 entries, 0 to 575
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               555 non-null    int64  
 1   NAME             555 non-null    object 
 2   INFO             555 non-null    object 
 3   LISTA_URL        555 non-null    object 
 4   REGULAR_PRICE    555 non-null    float64
 5   DISCOUNT_PRICE   555 non-null    float64
 6   PRODUCT          555 non-null    object 
 7   SLOGAN           460 non-null    object 
 8   CHARACTERISTICS  495 non-null    object 
 9   DESCRIPTION      555 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 47.7+ KB


C:\Users\Javier\AppData\Local\Temp\ipykernel_10948\766189877.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  productos_clean['CHARACTERISTICS'] = productos_clean['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
C:\Users\Javier\AppData\Local\Temp\ipykernel_10948\766189877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  productos_clean['CHARACTERISTICS'] = productos_clean['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
C:\Users\Javier\AppData\Local\Temp\ipykernel_10948\766189877.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' w

Reestructuramos el dataframe

In [38]:
col_1 = productos_clean.pop('PRODUCT')
col_2=productos_clean.pop('SLOGAN')
col_3=productos_clean.pop('DESCRIPTION')
col_4=productos_clean.pop('CHARACTERISTICS')

productos_clean.drop(columns=['NAME'],inplace=True)
productos_clean.drop(columns=['INFO'],inplace=True)

productos_clean.insert(loc= 1 , column= 'PRODUCT', value= col_1)
productos_clean.insert(loc= 2 , column= 'SLOGAN', value= col_2)
productos_clean.insert(loc= 3 , column= 'DESCRIPTION', value= col_3)
productos_clean.insert(loc= 4 , column= 'CHARACTERISTICS', value= col_4)
productos_clean.head()

C:\Users\Javier\AppData\Local\Temp\ipykernel_10948\3255870201.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  productos_clean.drop(columns=['NAME'],inplace=True)
C:\Users\Javier\AppData\Local\Temp\ipykernel_10948\3255870201.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  productos_clean.drop(columns=['INFO'],inplace=True)


,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,Desliz! Lubricante íntimo de agua 100ml,None,"Algunos lubricantes son un poco densos, otros ...",Bote de 100ml de venta exclusiva en amantis.ne...,https://www.amantis.net/desliz-lubricante-inti...,9.99,7.99
1,1,LIZO 2,Dildo de suave silicona en 3 tamaños,"En un azulejo de la cocina, en una puerta, en ...",Material: silicona médica de máxima calidadDim...,https://www.amantis.net/lizo-2-dildo-suave-sil...,59.99,17.99
2,2,FOXTAIL,plug anal cola de zorro de 35cm,Deja volar la imaginación y saca tu lado más s...,Tamaño plug S: 6.6cm Max Dia.: 2.8cmCola de pe...,https://www.amantis.net/foxtail-plug-anal-cola...,29.99,9.99
3,3,Vibrador Líquido con sabor Desliz! VIBRAGEL 30ml,hormigueo oral,¿Alguna vez has sentido un hormigueo en lo más...,"Ingredientes: agua, glicerina, hidroxietilcelu...",https://www.amantis.net/desliz-vibragel-liquid...,29.99,9.99
4,4,Bacanal FORTE TARRO 200ml,lubricante anal concentrado con aloe,Si alguna vez has pensado que el sexo anal es ...,Su textura no es grasa y no mancha. Compatible...,https://www.amantis.net/bacanal-forte-tarro-20...,24.99,24.99


#### Generamos ficheros .csv para ir guardando esta información antes de dejarlas salvadas en una BBDD Relacional.

In [39]:
productos_clean.to_csv('./Data/productos.csv',header=True,index=False)           # Tengo que generar el path correcto
comentarios_clean.to_csv('./Data/comentarios.csv',header=True,index=False)           # Tengo que generar el path correcto

## 2. Tratamiento de los datos de texto con RegEx y Librerías de NLP

Primeramente vamos a generar nuevas columnas a partir de *Name* y *Description*. 

Estas columnas las generaremos a partir de *RegEx* y los métodos *str.split* y *str.replace*.

Utilizaremos las siguientes librerías para el tratamiento  de la información.

In [3]:
import os
import pandas as pd
import numpy as np
import re
import spacy
from nltk.stem.snowball import SnowballStemmer

nlp = spacy.load('es_core_news_lg')

In [4]:
os.getcwd()
path='Data\scrapped_data.csv'
dataframe=pd.read_csv(path)
dataframe

,Name,Description,Price,Reduced Price,date,User,Ratings,Comment
0,"TOBOGANE HOT RABBIT, el superventas de amantis...",Vuelve nuestro vibrador de doble estimulación ...,21.99,39.99,"['martes 22 noviembre, 2022', 'jueves 07 julio...","['Rossi', 'Marina', 'Jennifer', 'Noa', 'Karen'...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","['Mi primera compra. Me encantó la textura, lo..."
1,"LIZO 2, Dildo de suave silicona en 3 tamaños","En un azulejo de la cocina, en una puerta, en ...",21.99,17.99,"['domingo 12 marzo, 2023', 'martes 10 enero, 2...","['Iria', 'Barney', 'Sara', 'Aida', 'Lucas', 'a...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",['Al no tener tope viene perfecto para usar co...
2,"FOXTAIL, plug anal cola de zorro de 35cm",Deja volar la imaginación y saca tu lado más s...,21.99,9.99,"['jueves 26 enero, 2023', 'jueves 05 enero, 20...","['Jonatan', 'Irene', 'Ainara', 'Ignacio', 'and...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4]","['Un buen plug, muy suave, aunque con el lubri..."
3,"TOBOGANE, el vibrador doble más vendido","Por favor, desabróchense los cinturones de seg...",21.99,36.99,"['lunes 06 marzo, 2023', 'sábado 04 marzo, 202...","['Teresa', 'Alicia', 'María', 'Sara', 'Raquel'...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",['Fue el primer vibrador que compré. Me lo rec...
4,"MENEO sube y baja, placer realista con control...","Si te gusta que te metan un buen meneo, hazte ...",21.99,44.99,"['miércoles 14 diciembre, 2022', 'miércoles 09...","['Francisco', 'Maria', 'Jose Javier', 'Carlos'...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",['Quería saber cuantos cm tiene la longitud qu...
...,...,...,...,...,...,...,...,...
571,JUSTISSE NECK - collar + bloqueadores de muñecas,"Muy a menudo, las parejas se quejan de que la ...",21.99,23.99,"['domingo 08 noviembre, 2020']",['Alejandro'],[5],['Es un poco engorroso a la hora de ponerlo so...
572,MS-VULVAN. Estimulador Total de Vulva (Cabezal...,La orgásmica colección de accesorios para acop...,21.99,8.99,"['jueves 02 agosto, 2018', 'martes 17 julio, 2...","['Inma', 'yulema', 'Cristina', 'Veturian', 'Pi...","[5, 5, 3, 5, 5, 4, 3]","['Es uno de mis cabezales estrella, es muy com..."
573,"AVATAR SIMON PRIDE, Orgulloso dildo de silicona",¿Quién dijo que en el mundo de los dildos no h...,21.99,34.99,"['martes 06 agosto, 2019']",['Javier'],[4],['A mi chica no le gustaba la típica aparienci...
574,"TSUNAMI FORMAS, Fundas de silicona para TSUNAM...",¿Has echado un vistazo a nuestro productazo la...,21.99,14.99,"['miércoles 11 noviembre, 2020', 'miércoles 01...","['Christian', 'Carmen', 'Meritxell', 'Esther',...","[5, 5, 4, 5, 5]",['Imprescindibles para disfrutar del Tsunami c...


### Columnas Product y Slogan.

En principio algunas de estas columnas son variables que no necesitaremos para estos estudios pero las guardaremos por si hay que hacer algún estudio posterior de las mismas (comparativas de productos similares de esta página web o de otras páginas web, por ejemplo).

In [5]:
dataframe['Name'] = dataframe['Name'].str.replace(r'-(?=\w)', '_')
dataframe[['Product', 'Slogan']] = dataframe['Name'].str.split('[,-.]', 1, expand=True)
dataframe['Product'] = dataframe['Product'].str.strip()
dataframe['Slogan'] = dataframe['Slogan'].str.strip()
dataframe

C:\Users\Javier\AppData\Local\Temp\ipykernel_8056\1963400927.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['Name'] = dataframe['Name'].str.replace(r'-(?=\w)', '_')
C:\Users\Javier\AppData\Local\Temp\ipykernel_8056\1963400927.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe[['Product', 'Slogan']] = dataframe['Name'].str.split('[,-.]', 1, expand=True)


,Name,Description,Price,Reduced Price,date,User,Ratings,Comment,Product,Slogan
0,"TOBOGANE HOT RABBIT, el superventas de amantis...",Vuelve nuestro vibrador de doble estimulación ...,21.99,39.99,"['martes 22 noviembre, 2022', 'jueves 07 julio...","['Rossi', 'Marina', 'Jennifer', 'Noa', 'Karen'...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","['Mi primera compra. Me encantó la textura, lo...",TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!
1,"LIZO 2, Dildo de suave silicona en 3 tamaños","En un azulejo de la cocina, en una puerta, en ...",21.99,17.99,"['domingo 12 marzo, 2023', 'martes 10 enero, 2...","['Iria', 'Barney', 'Sara', 'Aida', 'Lucas', 'a...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",['Al no tener tope viene perfecto para usar co...,LIZO 2,Dildo de suave silicona en 3 tamaños
2,"FOXTAIL, plug anal cola de zorro de 35cm",Deja volar la imaginación y saca tu lado más s...,21.99,9.99,"['jueves 26 enero, 2023', 'jueves 05 enero, 20...","['Jonatan', 'Irene', 'Ainara', 'Ignacio', 'and...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4]","['Un buen plug, muy suave, aunque con el lubri...",FOXTAIL,plug anal cola de zorro de 35cm
3,"TOBOGANE, el vibrador doble más vendido","Por favor, desabróchense los cinturones de seg...",21.99,36.99,"['lunes 06 marzo, 2023', 'sábado 04 marzo, 202...","['Teresa', 'Alicia', 'María', 'Sara', 'Raquel'...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",['Fue el primer vibrador que compré. Me lo rec...,TOBOGANE,el vibrador doble más vendido
4,"MENEO sube y baja, placer realista con control...","Si te gusta que te metan un buen meneo, hazte ...",21.99,44.99,"['miércoles 14 diciembre, 2022', 'miércoles 09...","['Francisco', 'Maria', 'Jose Javier', 'Carlos'...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",['Quería saber cuantos cm tiene la longitud qu...,MENEO sube y baja,placer realista con control remoto
...,...,...,...,...,...,...,...,...,...,...
571,JUSTISSE NECK - collar + bloqueadores de muñecas,"Muy a menudo, las parejas se quejan de que la ...",21.99,23.99,"['domingo 08 noviembre, 2020']",['Alejandro'],[5],['Es un poco engorroso a la hora de ponerlo so...,JUSTISSE NECK,collar + bloqueadores de muñecas
572,MS_VULVAN. Estimulador Total de Vulva (Cabezal...,La orgásmica colección de accesorios para acop...,21.99,8.99,"['jueves 02 agosto, 2018', 'martes 17 julio, 2...","['Inma', 'yulema', 'Cristina', 'Veturian', 'Pi...","[5, 5, 3, 5, 5, 4, 3]","['Es uno de mis cabezales estrella, es muy com...",MS_VULVAN,Estimulador Total de Vulva (Cabezal Masaje_Sex...
573,"AVATAR SIMON PRIDE, Orgulloso dildo de silicona",¿Quién dijo que en el mundo de los dildos no h...,21.99,34.99,"['martes 06 agosto, 2019']",['Javier'],[4],['A mi chica no le gustaba la típica aparienci...,AVATAR SIMON PRIDE,Orgulloso dildo de silicona
574,"TSUNAMI FORMAS, Fundas de silicona para TSUNAM...",¿Has echado un vistazo a nuestro productazo la...,21.99,14.99,"['miércoles 11 noviembre, 2020', 'miércoles 01...","['Christian', 'Carmen', 'Meritxell', 'Esther',...","[5, 5, 4, 5, 5]",['Imprescindibles para disfrutar del Tsunami c...,TSUNAMI FORMAS,Fundas de silicona para TSUNAMI POWER


### Separación entre Description y Characteristics.

Por un lado extraeremos las características en una nueva columna, dejando la Description sin los datos de la nueva columna. 

En principio algunas de estas columnas son variables que no necesitaremos para estos estudios pero las guardaremos por si hay que hacer algún estudio posterior de las mismas (comparativas de productos similares de esta página web o de otras páginas web, por ejemplo).

Dado que se observan que hay *\r* dentro de los textos de estas columnas, vamos a proceder a sustituir esto por un espacio.

In [6]:
dataframe['Characteristics'] = dataframe['Description'].str.split('Ver características y medidas', 1).str[1]
dataframe['Description'] = dataframe['Description'].str.split('Ver características y medidas', 1).str[0].str.strip()
dataframe['Characteristics'] = dataframe['Characteristics'].str.replace('\r', ' ')
dataframe['Description'] = dataframe['Description'].str.replace('\r', ' ')
dataframe

C:\Users\Javier\AppData\Local\Temp\ipykernel_8056\3655207862.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe['Characteristics'] = dataframe['Description'].str.split('Ver características y medidas', 1).str[1]
C:\Users\Javier\AppData\Local\Temp\ipykernel_8056\3655207862.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe['Description'] = dataframe['Description'].str.split('Ver características y medidas', 1).str[0].str.strip()


,Name,Description,Price,Reduced Price,date,User,Ratings,Comment,Product,Slogan,Characteristics
0,"TOBOGANE HOT RABBIT, el superventas de amantis...",Vuelve nuestro vibrador de doble estimulación ...,21.99,39.99,"['martes 22 noviembre, 2022', 'jueves 07 julio...","['Rossi', 'Marina', 'Jennifer', 'Noa', 'Karen'...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","['Mi primera compra. Me encantó la textura, lo...",TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,"Medidas: 19cm (11cm insertables) y 3,3cm/ 2,2c..."
1,"LIZO 2, Dildo de suave silicona en 3 tamaños","En un azulejo de la cocina, en una puerta, en ...",21.99,17.99,"['domingo 12 marzo, 2023', 'martes 10 enero, 2...","['Iria', 'Barney', 'Sara', 'Aida', 'Lucas', 'a...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",['Al no tener tope viene perfecto para usar co...,LIZO 2,Dildo de suave silicona en 3 tamaños,Material: silicona médica de máxima calidadDim...
2,"FOXTAIL, plug anal cola de zorro de 35cm",Deja volar la imaginación y saca tu lado más s...,21.99,9.99,"['jueves 26 enero, 2023', 'jueves 05 enero, 20...","['Jonatan', 'Irene', 'Ainara', 'Ignacio', 'and...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4]","['Un buen plug, muy suave, aunque con el lubri...",FOXTAIL,plug anal cola de zorro de 35cm,Tamaño plug S: 6.6cm Max Dia.: 2.8cmCola de pe...
3,"TOBOGANE, el vibrador doble más vendido","Por favor, desabróchense los cinturones de seg...",21.99,36.99,"['lunes 06 marzo, 2023', 'sábado 04 marzo, 202...","['Teresa', 'Alicia', 'María', 'Sara', 'Raquel'...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",['Fue el primer vibrador que compré. Me lo rec...,TOBOGANE,el vibrador doble más vendido,"Vibrador con doble motor. Medidas: 19cm, (11c..."
4,"MENEO sube y baja, placer realista con control...","Si te gusta que te metan un buen meneo, hazte ...",21.99,44.99,"['miércoles 14 diciembre, 2022', 'miércoles 09...","['Francisco', 'Maria', 'Jose Javier', 'Carlos'...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",['Quería saber cuantos cm tiene la longitud qu...,MENEO sube y baja,placer realista con control remoto,Dildo percutor y vibrador con control remoto3 ...
...,...,...,...,...,...,...,...,...,...,...,...
571,JUSTISSE NECK - collar + bloqueadores de muñecas,"Muy a menudo, las parejas se quejan de que la ...",21.99,23.99,"['domingo 08 noviembre, 2020']",['Alejandro'],[5],['Es un poco engorroso a la hora de ponerlo so...,JUSTISSE NECK,collar + bloqueadores de muñecas,Características: • Kit compuesto por un collar...
572,MS_VULVAN. Estimulador Total de Vulva (Cabezal...,La orgásmica colección de accesorios para acop...,21.99,8.99,"['jueves 02 agosto, 2018', 'martes 17 julio, 2...","['Inma', 'yulema', 'Cristina', 'Veturian', 'Pi...","[5, 5, 3, 5, 5, 4, 3]","['Es uno de mis cabezales estrella, es muy com...",MS_VULVAN,Estimulador Total de Vulva (Cabezal Masaje_Sex...,· Superficie de contacto de 8cm x 4.5cm · Rea...
573,"AVATAR SIMON PRIDE, Orgulloso dildo de silicona",¿Quién dijo que en el mundo de los dildos no h...,21.99,34.99,"['martes 06 agosto, 2019']",['Javier'],[4],['A mi chica no le gustaba la típica aparienci...,AVATAR SIMON PRIDE,Orgulloso dildo de silicona,Juguete de silicona 100%Dimensiones: 20 cm de ...
574,"TSUNAMI FORMAS, Fundas de silicona para TSUNAM...",¿Has echado un vistazo a nuestro productazo la...,21.99,14.99,"['miércoles 11 noviembre, 2020', 'miércoles 01...","['Christian', 'Carmen', 'Meritxell', 'Esther',...","[5, 5, 4, 5, 5]",['Imprescindibles para disfrutar del Tsunami c...,TSUNAMI FORMAS,Fundas de silicona para TSUNAMI POWER,Hechas en silicona de grado médicoSin ftalatos...


Eliminamos *Nombre* y sustituimos por *Product*, *Slogan*, *Description* y *Characteristics*.

In [7]:
col_1 = dataframe.pop('Product')
col_2=dataframe.pop('Slogan')
col_3=dataframe.pop('Characteristics')
dataframe.drop(columns=['Name'],inplace=True)

dataframe.insert(loc= 0 , column= 'Product', value= col_1)
dataframe.insert(loc= 1 , column= 'Slogan', value= col_2)
dataframe.insert(loc= 3 , column= 'Characteristics', value= col_3)
dataframe.head()

,Product,Slogan,Description,Characteristics,Price,Reduced Price,date,User,Ratings,Comment
0,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,Vuelve nuestro vibrador de doble estimulación ...,"Medidas: 19cm (11cm insertables) y 3,3cm/ 2,2c...",21.99,39.99,"['martes 22 noviembre, 2022', 'jueves 07 julio...","['Rossi', 'Marina', 'Jennifer', 'Noa', 'Karen'...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","['Mi primera compra. Me encantó la textura, lo..."
1,LIZO 2,Dildo de suave silicona en 3 tamaños,"En un azulejo de la cocina, en una puerta, en ...",Material: silicona médica de máxima calidadDim...,21.99,17.99,"['domingo 12 marzo, 2023', 'martes 10 enero, 2...","['Iria', 'Barney', 'Sara', 'Aida', 'Lucas', 'a...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",['Al no tener tope viene perfecto para usar co...
2,FOXTAIL,plug anal cola de zorro de 35cm,Deja volar la imaginación y saca tu lado más s...,Tamaño plug S: 6.6cm Max Dia.: 2.8cmCola de pe...,21.99,9.99,"['jueves 26 enero, 2023', 'jueves 05 enero, 20...","['Jonatan', 'Irene', 'Ainara', 'Ignacio', 'and...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4]","['Un buen plug, muy suave, aunque con el lubri..."
3,TOBOGANE,el vibrador doble más vendido,"Por favor, desabróchense los cinturones de seg...","Vibrador con doble motor. Medidas: 19cm, (11c...",21.99,36.99,"['lunes 06 marzo, 2023', 'sábado 04 marzo, 202...","['Teresa', 'Alicia', 'María', 'Sara', 'Raquel'...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",['Fue el primer vibrador que compré. Me lo rec...
4,MENEO sube y baja,placer realista con control remoto,"Si te gusta que te metan un buen meneo, hazte ...",Dildo percutor y vibrador con control remoto3 ...,21.99,44.99,"['miércoles 14 diciembre, 2022', 'miércoles 09...","['Francisco', 'Maria', 'Jose Javier', 'Carlos'...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",['Quería saber cuantos cm tiene la longitud qu...


#### Generación de tags.

Una vez creadas estas nuevas columnas vamos a generar los tags de cada uno de los parámetros a considerar a través de una serie de listas.

Para esto vamos a eliminar las mayúsculas y los acentos en los textos pertinentes con Regex y pasaremos a lemmatizar las palabras para poder trabajar con ellas indistitantemente.

Dejaremos para estos procesos unicamente las cuatro primeras columnas, generando un dataframe nuevo para el tratamiento de los datos relevantes.

In [9]:
df_engineer=dataframe.iloc[:,:3]
df_engineer

,Product,Slogan,Description
0,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,Vuelve nuestro vibrador de doble estimulación ...
1,LIZO 2,Dildo de suave silicona en 3 tamaños,"En un azulejo de la cocina, en una puerta, en ..."
2,FOXTAIL,plug anal cola de zorro de 35cm,Deja volar la imaginación y saca tu lado más s...
3,TOBOGANE,el vibrador doble más vendido,"Por favor, desabróchense los cinturones de seg..."
4,MENEO sube y baja,placer realista con control remoto,"Si te gusta que te metan un buen meneo, hazte ..."
...,...,...,...
571,JUSTISSE NECK,collar + bloqueadores de muñecas,"Muy a menudo, las parejas se quejan de que la ..."
572,MS_VULVAN,Estimulador Total de Vulva (Cabezal Masaje_Sex...,La orgásmica colección de accesorios para acop...
573,AVATAR SIMON PRIDE,Orgulloso dildo de silicona,¿Quién dijo que en el mundo de los dildos no h...
574,TSUNAMI FORMAS,Fundas de silicona para TSUNAMI POWER,¿Has echado un vistazo a nuestro productazo la...


Y las siguientes funciones para utilizar con la función *apply*.

In [ ]:
def extraer_lemas(texto):
    doc = nlp(texto)
    lemas = [token.lemma_ for token in doc if token.is_alpha]
    return lemas

def remove_accents(text):
    pattern = '[áéíóúÁÉÍÓÚ]'
    replace = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
    return re.sub(pattern, lambda match: replace[match.group()], text)

def spanish_stemmer(x):
    stemmer = SnowballStemmer('spanish')
    return " ".join([stemmer.stem(word) for word in x.split()])


Generaremos los #tags con las siguientes listas. Estos tags serán nuevas columnas en el dataframe.

In [ ]:
juguetes=['dildo','plug','vibrador','masturbador']
BDSM=['latex','bdsm','arnes','strap','cera','ligadura','cuerda','cuero','sumision','dominacion','latigo']
muebles=['columpio','sillon']
lenceria=['body','panties','arnes',]
anal=['anal']
masculino=['hombre','masculino']
femenino=['mujer','femenino']
